<a href="https://colab.research.google.com/github/RinoGeek/DiabeteX-Predictor/blob/main/GesturaNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, MaxPooling2D, Conv2D, Flatten, Dropout
from keras.utils import to_categorical
from PIL.Image import Resampling
from keras.utils import load_img, img_to_array

In [8]:
#!unzip /content/drive/MyDrive/other-code-computer-vision/brail.zip

In [9]:
train = "/content/brail/train"
test = "/content/brail/test"

In [10]:
def load_data(data_dir):
    X = []
    y = []
    # Initialiser le label encoder
    label_encoder = LabelEncoder()
    img_width, img_height = 250, 250

    # Parcourir les sous-répertoires dans le répertoire de données principal
    for class_dir in os.listdir(data_dir):
        if os.listdir:
            # Construire le chemin vers le sous-répertoire de classe actuel
            class_path = os.path.join(data_dir, class_dir)

            # Encoder le nom de la classe en utilisant le label encoder
            class_label = label_encoder.fit_transform([class_dir])[0]

        # Parcourir les images dans le sous-répertoire de classe actuel
        for img_name in os.listdir(class_path):
            # Construire le chemin vers l'image actuelle
            img_path = os.path.join(class_path, img_name)
            # Charger l'image à partir du chemin
            img = load_img(img_path, color_mode="grayscale")
            img = img.resize((img_width, img_height), resample=Resampling.BICUBIC)
            img_array = img_to_array(img)

            # Remodeler le tableau d'image pour correspondre à la forme d'entrée attendue pour le modèle
            img_array = img_array.reshape(img_width, img_height, 1).astype('float32')
            X.append(img_array)  # Ajouter le tableau d'image à la liste des images
            y.append(class_label)  # Ajouter l'étiquette encodée à la liste des étiquettes
    return np.array(X), np.array(y)  # Convertir les listes en tableaux NumPy et les retourner


In [ ]:
xtrain, ytrain = load_data(train)

In [ ]:
xtest, ytest = load_data(test)

In [ ]:
xtrain /= 255
xtest /= 255

In [ ]:
# Créer un modèle séquentiel
model = Sequential()
img_width, img_height = 250, 250
# Ajouter une couche de convolution 2D avec 32 filtres, une taille de noyau de 3x3 et une fonction d'activation ReLU
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))

# Ajouter une couche de max pooling 2D
model.add(MaxPooling2D((2, 2)))

# Ajouter une autre couche de convolution 2D avec 64 filtres et une taille de noyau de 3x3
model.add(Conv2D(64, (3, 3), activation='relu'))

# Ajouter une autre couche de max pooling 2D
model.add(MaxPooling2D((2, 2)))

# Ajouter une autre couche de convolution 2D avec 128 filtres et une taille de noyau de 3x3
model.add(Conv2D(128, (3, 3), activation='relu'))

# Ajouter une autre couche de max pooling 2D
model.add(MaxPooling2D((2, 2)))

# Aplatir les données pour les passer à travers des couches entièrement connectées
model.add(Flatten())

# Ajouter une couche entièrement connectée avec 512 neurones et une fonction d'activation ReLU
model.add(Dense(512, activation='relu'))

# Ajouter une couche de dropout pour éviter le surapprentissage
model.add(Dropout(0.5))

# Ajouter une couche de sortie avec un nombre d'unités égal au nombre de classes et une fonction d'activation softmax
model.add(Dense(29, activation='softmax'))

# Compiler le modèle en spécifiant la fonction de perte, l'optimiseur et les métriques à surveiller
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Afficher un résumé de l'architecture du modèle
model.summary()
